In [1]:
from src.components.transform import Transform
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('test').getOrCreate()

t = Transform(spark)

In [2]:
fo = t.transform()

In [3]:
fo.show()

+--------------------+---------------------+---------------------+--------------------------+--------------------+--------------+--------------------+------------------------+-------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+-------------+------------------------+------------+--------------------+------------------+------------+-------------+-----+---------------------+--------------------------+-----------------+--------------------+-----------------+-------------------+------------------+----------------+----------------+-----------------------+----------------------+--------------------+--------------------+--------------------+------------+--------------+-------------------+-------------------+--------------------+------------+----------------------+-------------------+
|       customer_city|customer_geo_lat_mean|customer_geo_lng_mean|customer_geolocation_state|         customer_id|custome

In [5]:
from pyspark.sql.functions import col

for column in fo.columns:
    na_counts = fo.select(column).filter(col(column).isNull()).count()

    print(f"Column: {column} -- NA {na_counts} \n")

Column: customer_city -- NA 0 

Column: customer_geo_lat_mean -- NA 317 

Column: customer_geo_lng_mean -- NA 317 

Column: customer_geolocation_state -- NA 317 

Column: customer_id -- NA 0 

Column: customer_state -- NA 0 

Column: customer_unique_id -- NA 0 

Column: customer_zip_code_prefix -- NA 0 

Column: freight_value -- NA 0 

Column: order_approved_at -- NA 15 

Column: order_delivered_carrier_date -- NA 1254 

Column: order_delivered_customer_date -- NA 2588 

Column: order_estimated_delivery_date -- NA 0 

Column: order_id -- NA 0 

Column: order_item_id -- NA 0 

Column: order_purchase_timestamp -- NA 0 

Column: order_status -- NA 0 

Column: payment_installments -- NA 3 

Column: payment_sequential -- NA 3 

Column: payment_type -- NA 3 

Column: payment_value -- NA 3 

Column: price -- NA 0 

Column: product_category_name -- NA 1709 

Column: product_description_lenght -- NA 1709 

Column: product_height_cm -- NA 0 

Column: product_id -- NA 0 

Column: product_length_c

In [6]:
fo.filter(col('customer_geo_lat_mean').isNull()).show()

+-------------+---------------------+---------------------+--------------------------+--------------------+--------------+--------------------+------------------------+-------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+-------------+------------------------+------------+--------------------+------------------+------------+-------------+------+---------------------+--------------------------+-----------------+--------------------+-----------------+-------------------+------------------+----------------+----------------+-----------------------+----------------------+--------------------+--------------------+--------------------+------------+--------------------+-------------------+-------------------+--------------------+------------+----------------------+-------------------+
|customer_city|customer_geo_lat_mean|customer_geo_lng_mean|customer_geolocation_state|         customer_id|customer_state